# Python Play: bring your own plot day

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import netCDF4 as nc
import os
from itertools import compress
import datetime as dt
from datetime import date
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib import cm
from math import sin, cos, sqrt, atan2, radians
from matplotlib import rc
from IPython.display import HTML
import glob
from IPython.display import Image

%matplotlib inline

## Option 1: wind streamlines & ocean surface

In [ ]:
def plot_5d_mackenzie(ax, fday, time, files_ref, overlap_dates, overlap_uwind, overlap_vwind, wind=False, lw=0, \
                     set_lw=False):

    proj2 = Basemap(projection='stere',lat_0 = 71, lon_0 = -135, resolution='i', width=1.5e6, height=1.0e6, ax=ax)
    
    x_land, y_land = proj2(lon, lat)
    proj2.drawcoastlines(color='#373736', linewidth=1.8)
    proj2.fillcontinents(color=land_color, lake_color=land_color, zorder=5);
    proj2.drawparallels(np.arange(int(60),int(90),2),labels=[1,0,0,0], linewidth=0.0, fontsize=10)
    proj2.drawmeridians(np.arange(int(-150),int(-30),10),labels=[0,0,0,1], linewidth=0.0, fontsize=10)

    #--------------------- five day surface continental runoff field------------------------------
    x_model, y_model = proj2(lons, lats)

    CS2 = proj2.contourf(x_model, y_model, fday, \
                         20, cmap=cont_cmap, vmin=0.01, vmax=0.15, extend='both')

    ax.set_title(files_ref[time][93:97]+'-'+files_ref[time][97:99]+'-'+files_ref[time][99:101]+' to '\
                 +files_ref[time][102:106]+'-'+files_ref[time][106:108]+'-'+files_ref[time][108:110])

    #--------------------- wind stream plot ------------------------------------------------------
    if wind:
        startt = np.datetime64(dt.datetime.strptime(files_ref[time][93:101],  '%Y%m%d'))
        endt   = np.datetime64(dt.datetime.strptime(files_ref[time][102:110], '%Y%m%d'))
        
        time_ind = np.array(np.where((overlap_dates > startt) & (overlap_dates < endt)))[0]
        
        u_1h_wind = np.zeros(overlap_uwind[0,:,:].shape); v_1h_wind=np.zeros(overlap_vwind[0,:,:].shape);
        for t in time_ind:
            u_1h_wind = np.add(overlap_uwind[t,:,:], u_1h_wind)
            v_1h_wind = np.add(overlap_vwind[t,:,:], v_1h_wind)
            
        u_5d_wind = u_1h_wind/len(time_ind)
        v_5d_wind = v_1h_wind/len(time_ind)
        
        speed = np.sqrt(u_5d_wind**2 + v_5d_wind**2)
        
        wlons, wlats   = np.meshgrid(wind_lons, wind_lats)
        x_wind, y_wind = proj2(wlons, wlats)
        
        # Streamline thickness is related to speed and set during the first time period, so that it is
        # the same for all plots
        if time==0:
            lw = 4*speed / speed.max()
        elif set_lw:
            lw = 6*speed / speed.max()
        
        ax.streamplot(x_wind[0,:], y_wind[:,0], u_5d_wind, v_5d_wind, \
                      arrowsize=2.5, density=1, color='#323232', linewidth=lw, arrowstyle='->')


    proj2.drawmapscale(-147, 67.5, -140, 75, 400, barstyle='fancy', zorder=5, fontsize=12)
    return lw

## Option 2: pretty basemap contour plots

## Option 3: making colormaps with colorblindness in mind

## Option 4: showing sparse observations and model results on a transect plot with inset